# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [2]:
! pip install cryptography

You should consider upgrading via the 'c:\users\diebl\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [4]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

engine = create_engine(connection_string)


I created the May table using the query below with Python. 
I changed the type of the cell to avoid the error "table already created" I got when I worked on the notebook the day after.


query = '''  
    CREATE TABLE rental_may (
    `rental_id` int UNIQUE NOT NULL,
  `rental_date` date DEFAULT NULL,
  `inventory_id` mediumint UNIQUE NOT NULL,
  `customer_id` smallint UNIQUE NOT NULL,
  `return_date`date DEFAULT NULL,
  `staff_id` tinyint UNIQUE NOT NULL,
  `last_update`timestamp DEFAULT NULL,
    CONSTRAINT PRIMARY KEY (rental_id) );
    '''



## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

done on WB using this query :

INSERT IGNORE INTO rental_may
SELECT* FROM rental
WHERE MONTH(rental_date) = 05;

In [5]:
rental_may = pd.read_sql_query('SELECT * FROM rental_may', engine)
rental_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24,367,130,2005-05-26,1,2006-02-15 21:30:53
1,4,2005-05-24,2452,333,2005-06-03,2,2006-02-15 21:30:53


## Create a table `rentals_june` to store the data from rental table with information for the month of June.


I don't know why the below query that has worked to create the rental_may table has not worked for the rental_june table. So I have done it on WB :


query = '''  
    CREATE TABLE rental_june (
    `rental_id` int UNIQUE NOT NULL,
  `rental_date` date DEFAULT NULL,
  `inventory_id` mediumint UNIQUE NOT NULL,
  `customer_id` smallint UNIQUE NOT NULL,
  `return_date`date DEFAULT NULL,
  `staff_id` tinyint NOT NULL,
  `last_update`timestamp DEFAULT NULL,
    CONSTRAINT PRIMARY KEY (rental_id) );
    '''

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

Done on WB using this query :

    INSERT IGNORE INTO rental_june, 
    SELECT *
    FROM rental 
    WHERE MONTH(rental_date) = 06;
    


In [6]:
rental_june= pd.read_sql_query('SELECT * FROM rental_june', engine)
rental_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14,1632,416,2005-06-18,2,2006-02-15 21:30:53
1,1159,2005-06-14,4395,516,2005-06-17,1,2006-02-15 21:30:53
2,1160,2005-06-14,2795,239,2005-06-18,2,2006-02-15 21:30:53
3,1161,2005-06-14,1690,285,2005-06-21,1,2006-02-15 21:30:53
4,1162,2005-06-14,987,310,2005-06-23,1,2006-02-15 21:30:53


## Check the number of rentals for each customer for May.


In [30]:
query = '''
        SELECT customer_id, COUNT(rental_id) AS may
        FROM rental_may
        GROUP BY customer_id;  
        '''

rental_may1 = pd.read_sql_query(query, engine)
rental_may1.head()


,customer_id,may
0,130,1
1,333,1


## Check the number of rentals for each customer for June.
 

In [31]:
query = '''
        SELECT customer_id, COUNT(rental_id) AS june
        FROM rental_june
        GROUP BY customer_id
        ORDER BY customer_id;  
        '''

rental_june2 = pd.read_sql_query(query, engine)
rental_june2.head()

,customer_id,june
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [32]:
df_merge1 = pd.merge(rental_may1, rental_june2, on='customer_id')
df_merge1

,customer_id,may,june
0,130,1,1
1,333,1,1


I didn't manage to write the function to make the comparison. Could you please provide me with the code? thanks in advance.